In [ ]:
import numpy as np
import copy
np.set_printoptions(linewidth =np.inf,threshold=np.inf)
import networkx as nx

class IntCode:
    def __init__(self, sequence):
        self.sequence = {index:val for index, val in enumerate(sequence.copy())}
        self.index = 0
        self.relative_base = 0
        self.input_val = []
    
    def reset(self):
        self.index = 0
        self.relative_base = 0
        self.input_val = []

    @staticmethod
    def read_mod(mods, offset):
            return mods[offset] if 0 <= offset < len(mods) else 0
    
    def read_val(self, mods, offset):
        val = self.sequence.get(self.index + offset, 0)

        mod = self.read_mod(mods, offset)
        if mod == 0:
            return self.sequence.get(val, 0)
        elif mod == 1:
            return val
        elif mod == 2:
            return self.sequence.get(self.relative_base + val, 0)

    def write_val(self, new_value, mods, offset):
        val = self.sequence.get(self.index + offset, 0)

        mod = self.read_mod(mods, offset)
        if mod == 0:
            self.sequence.update({val: new_value})
        elif mod == 2:
            self.sequence.update({self.relative_base + val: new_value})
    
    def run(self):
        while(True):
            opcode = self.sequence[self.index]
            val = opcode % 100
            mods = list(map(int, str(opcode)))[:-2]
            mods.reverse()
            self.index += 1

            if val in [1, 2, 7, 8]:
                op1 = self.read_val(mods, 0)
                op2 = self.read_val(mods, 1)
                if val == 1:
                    res = op1 + op2
                elif val == 2:
                    res = op1 * op2
                elif val == 7:
                    res = 1 if op1 < op2 else 0
                elif val == 8:
                    res = 1 if op1 == op2 else 0

                self.write_val(res, mods, 2)
                self.index += 3

            elif val == 3:
                if not self.input_val:
                    self.index -= 1 # since we added one earlier, we need to remove it before breaking
                    break
                    
                input_val = self.input_val.pop(0)
                self.write_val(input_val, mods, 0)

                self.index += 1

            elif val in [4, 9]:
                op1 = self.read_val(mods, 0)
                if val == 4:
                    yield op1
                elif val == 9:
                    self.relative_base += op1

                self.index += 1

            elif val in [5, 6, 9]:
                op1 = self.read_val(mods, 0)

                if val == 5 and op1 or val == 6 and not op1:
                    self.index = self.read_val(mods, 1)
                else:
                    self.index += 2

            elif val == 99:
                print("seq finit!")
                break

            else:
                raise ValueError("RIP")

class KevinRobot:
    def __init__(self, seq):
        self.seq = seq
        self.intcode = IntCode(seq)
        self.mapping = {1:"north", 2:"south", 3:"west", 4:"est"}
        self.reverse_mapping = {v: k for k, v in self.mapping.items()}
        self.direction_symbole_mapping = {"north": "^", "est": ">", "south": "v", "west": "<"}
        self.direction = "north"
        self.position = (0, 0)
        self.symbole_mapping = {0:"#", 1:"~", 2:"*", 3:"X"}
        self.canevas = {self.position:3}
        self.reset()
        self.G = nx.Graph()
        self.G.add_node(self.position)
        
    def rotate_right(self):
        if self.direction == "north":
            self.direction = "est"
        elif self.direction == "est":
            self.direction = "south"
        elif self.direction == "south":
            self.direction = "west"
        elif self.direction == "west":
            self.direction = "north"

    def rotate_left(self):
        if self.direction == "north":
            self.direction = "west"
        elif self.direction == "west":
            self.direction = "south"
        elif self.direction == "south":
            self.direction = "est"
        elif self.direction == "est":
            self.direction = "north"
    
    def move(self):
        if self.direction == "north":
            diff =  (-1, 0)
        elif self.direction == "est":
            diff =  (0, 1)
        elif self.direction == "south":
            diff = (1, 0)
        elif self.direction == "west":
            diff = (0, -1)
        
        self.position = tuple([sum(x) for x in zip(self.position, diff)])

    def reset(self):
        self.position = (0, 0)
        self.direction = "north"
        self.intcode = IntCode(self.seq)
        
    def _map(self):
        min_x = min(self.canevas, key=lambda k: k[0])[0]
        max_x = max(self.canevas, key=lambda k: k[0])[0]
        min_y = min(self.canevas, key=lambda k: k[1])[1]
        max_y = max(self.canevas, key=lambda k: k[1])[1]

        offset = (abs(min_x), abs(min_y))
        canevas = np.full((max_x - min_x + 1, max_y - min_y + 1), " ", dtype=str)

        for index, value in self.canevas.items():
            symbol = self.symbole_mapping[value]
            canevas[tuple([sum(x) for x in zip(offset, index)])] = symbol
            
        canevas[tuple([sum(x) for x in zip(offset, self.position)])] = self.direction_symbole_mapping[self.direction]
        
        print(canevas)
    
    def part_1(self):
        previous_position = self.position
       
        found_left = False
        found_right = False
        print("always_left")
        while(not found_left or not found_right):
            self.intcode.input_val.append(self.reverse_mapping[self.direction])
            self.move()
            
            output = list(self.intcode.run())[0]

            self.canevas[self.position] = output        
            
            if output == 0:
                self.position = previous_position
                if not found_left:
                    self.rotate_right()
                else:
                    self.rotate_left()
            
            elif output == 1:
                self.G.add_node(self.position)
                self.G.add_edge(previous_position, self.position)
                if not found_left:
                    self.rotate_left()
                else: 
                    self.rotate_right()
                previous_position = self.position
                
            elif output == 2:
                self.G.add_node(self.position)
                self.G.add_edge(previous_position, self.position)
                if not found_left:
                    self._map()
                    found_left = True
                    self.reset()
                    previous_position = (0, 0)
                    print("always right")
                else:
                    self._map()
                    found_right = True
                    self.oxygen = self.position
                    return len(nx.shortest_path(kg.G, (0, 0), self.oxygen)) - 1
                    
    
    def part_2(self):
        if nx.is_empty(self.G):
            self.part_1()
        
        return nx.dag_longest_path_length(nx.bfs_tree(self.G, self.oxygen))
        
        

In [ ]:
seq = [3,1033,1008,1033,1,1032,1005,1032,31,1008,1033,2,1032,1005,1032,58,1008,1033,3,1032,1005,1032,81,1008,1033,4,1032,1005,1032,104,99,101,0,1034,1039,101,0,1036,1041,1001,1035,-1,1040,1008,1038,0,1043,102,-1,1043,1032,1,1037,1032,1042,1106,0,124,1002,1034,1,1039,1001,1036,0,1041,1001,1035,1,1040,1008,1038,0,1043,1,1037,1038,1042,1105,1,124,1001,1034,-1,1039,1008,1036,0,1041,1002,1035,1,1040,1001,1038,0,1043,101,0,1037,1042,1105,1,124,1001,1034,1,1039,1008,1036,0,1041,101,0,1035,1040,102,1,1038,1043,1001,1037,0,1042,1006,1039,217,1006,1040,217,1008,1039,40,1032,1005,1032,217,1008,1040,40,1032,1005,1032,217,1008,1039,1,1032,1006,1032,165,1008,1040,7,1032,1006,1032,165,1101,2,0,1044,1105,1,224,2,1041,1043,1032,1006,1032,179,1102,1,1,1044,1106,0,224,1,1041,1043,1032,1006,1032,217,1,1042,1043,1032,1001,1032,-1,1032,1002,1032,39,1032,1,1032,1039,1032,101,-1,1032,1032,101,252,1032,211,1007,0,45,1044,1106,0,224,1102,1,0,1044,1105,1,224,1006,1044,247,101,0,1039,1034,1001,1040,0,1035,102,1,1041,1036,1002,1043,1,1038,1001,1042,0,1037,4,1044,1105,1,0,20,12,24,92,28,41,2,48,89,3,20,28,54,25,52,5,1,6,33,88,74,9,9,37,88,28,76,41,47,37,36,57,47,29,66,5,85,31,41,36,91,73,35,57,47,84,35,24,73,58,46,6,12,33,71,36,91,84,10,11,63,23,54,49,36,43,17,37,67,92,8,90,27,35,73,21,43,93,43,23,73,13,21,92,17,93,9,82,29,43,75,91,64,28,78,83,6,5,87,81,44,44,25,64,36,90,89,39,50,1,99,8,46,61,82,44,30,92,83,27,9,58,96,4,48,22,98,22,14,58,11,36,98,14,71,29,63,95,23,70,74,20,97,35,96,18,29,68,20,69,39,56,2,37,82,15,34,29,88,86,11,13,75,1,73,48,59,71,44,42,83,89,17,53,82,1,70,35,79,28,82,62,2,62,8,79,11,20,27,50,6,77,47,27,4,24,64,37,22,84,27,49,40,13,98,25,28,98,94,18,10,40,95,6,27,11,54,43,30,53,5,72,73,92,44,30,61,9,97,84,18,30,65,17,34,75,86,47,1,32,14,70,32,27,84,65,63,37,57,90,25,64,7,54,76,29,94,33,53,29,58,21,3,81,88,50,16,53,24,28,96,64,12,36,67,13,33,67,78,43,90,20,46,31,44,87,30,35,85,94,22,86,12,63,92,6,43,24,47,26,64,77,39,21,76,9,63,79,17,34,61,4,1,19,63,89,30,85,19,95,58,91,16,97,35,50,81,3,59,37,96,17,79,12,46,81,9,64,47,10,48,25,64,2,62,69,23,32,71,77,41,28,65,98,7,39,76,31,61,41,18,56,39,80,95,24,41,38,97,29,32,65,42,97,10,91,68,5,27,55,35,94,4,10,69,22,40,2,81,5,88,1,99,3,99,75,7,87,60,39,26,53,14,20,80,94,2,49,19,79,41,46,42,9,82,13,51,76,19,75,18,28,89,56,21,92,86,17,58,17,30,50,19,34,47,71,1,93,21,36,90,77,40,20,80,63,17,7,52,79,10,53,64,24,40,4,64,24,39,77,55,31,29,91,77,46,36,15,44,96,22,19,98,76,2,20,9,99,76,2,87,84,31,47,3,16,95,84,4,32,13,56,34,79,93,18,89,92,12,92,80,33,78,42,76,33,14,42,64,81,5,54,15,92,97,56,29,5,63,21,6,76,58,65,28,29,58,18,73,49,25,95,59,40,59,5,15,72,36,62,43,77,1,20,48,31,90,22,63,21,79,31,75,24,21,64,84,16,65,91,38,35,29,57,72,61,73,5,35,94,36,16,66,17,88,56,6,41,75,6,25,87,27,68,42,23,66,19,21,76,2,33,92,21,76,44,30,79,42,46,63,59,41,94,20,66,3,71,60,54,82,2,17,98,38,90,95,3,15,65,53,39,92,6,20,62,53,33,12,52,92,39,60,72,41,86,16,40,25,63,14,21,32,24,10,68,97,38,33,40,97,93,43,40,1,94,84,27,23,71,9,68,32,19,15,25,71,57,10,52,25,92,12,72,90,42,97,79,4,73,83,25,80,26,68,35,8,91,47,43,15,57,76,68,37,29,92,92,24,52,53,37,26,94,23,49,18,20,63,38,5,15,77,66,39,89,14,20,19,80,15,63,81,3,60,74,13,33,85,71,94,7,18,95,75,34,73,23,28,99,35,77,60,71,37,74,43,50,46,55,28,97,16,90,21,60,89,88,52,48,39,72,3,46,43,77,17,79,20,71,41,67,26,99,13,54,90,64,20,75,0,0,21,21,1,10,1,0,0,0,0,0,0]
kg = KevinRobot(seq)

In [ ]:
kg.part_1()

In [ ]:
kg.part_2()